In [9]:
from src.algorithms.alternating_least_squares import AlternatingLeastSquares
from src.helpers.dataset_indexer import DatasetIndexer
from src.helpers.checkpoint_manager import CheckpointManager
from src.recommenders import CollaborativeFilteringRecommenderBuilder
from src.backends import Backend

from src.settings import settings

In [10]:
dataset_indexer = DatasetIndexer(
    file_path="./ml-32m/ratings.csv",
    user_header="userId",
    item_header="movieId",
    rating_header="rating",
    limit=10000,
)

indexed_data = dataset_indexer.index(approximate_train_ratio=0.8)

The current implementation does not split the data into train and test sets exactly with the provided ratio. We use the provided ratio as a probability for a Bernoulli distribution to know whether the data point should be used as a training data or a test data.
Limit of lines (.i.e 10000) to index has been reached. Exiting without loading the rest... 


In [11]:
alternating_least_squares = AlternatingLeastSquares(
    hyper_lambda=settings.als.HYPER_LAMBDA,
    hyper_gamma=settings.als.HYPER_GAMMA,
    hyper_tau=settings.als.HYPER_TAU,
    hyper_n_epochs=settings.als.HYPER_N_EPOCH,
    hyper_n_factors=settings.als.HYPER_N_FACTOR,
)

als_backend = Backend(
    # Define the algorithm
    algorithm=alternating_least_squares,
    checkpoint_manager=CheckpointManager(
        checkpoint_folder=settings.als.CHECKPOINT_FOLDER,
    ),
)

In [12]:
recommender_builder = CollaborativeFilteringRecommenderBuilder(
    backend=als_backend,
)

recommender = recommender_builder.build(data=indexed_data)

# recommenders.recommend(None)

target =>  LearningTargetEnum.USER
_get_other_target_id =>  <function AlternatingLeastSquares.fit.<locals>.<lambda> at 0x70e4ec28dbd0>
other_target_factors =>  (3965, 10)
other_target_id =>  0
other_target_factors[other_target_id] =>  (10,)
other_target_id =>  1
other_target_factors[other_target_id] =>  (10,)
other_target_id =>  2
other_target_factors[other_target_id] =>  (10,)
other_target_id =>  3
other_target_factors[other_target_id] =>  (10,)
other_target_id =>  4
other_target_factors[other_target_id] =>  (10,)
other_target_id =>  5
other_target_factors[other_target_id] =>  (10,)
other_target_id =>  6
other_target_factors[other_target_id] =>  (10,)
other_target_id =>  7
other_target_factors[other_target_id] =>  (10,)
other_target_id =>  8
other_target_factors[other_target_id] =>  (10,)
other_target_id =>  9
other_target_factors[other_target_id] =>  (10,)
other_target_id =>  10
other_target_factors[other_target_id] =>  (10,)
other_target_id =>  11
other_target_factors[other_target_i

ValueError: shapes (10,) and (1,66,10) not aligned: 10 (dim 0) != 66 (dim 1)

Method:  numpy


KeyboardInterrupt: 